### Install some libs

!pip install treelib

### Import some libs

In [1]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np
import regex as re
import subprocess

### Initialize Tree-sitter go parser

In [2]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

### Define the commit for each year

See track_ft_changes_kops.ipynb for the methodology on how to find these commits

In [3]:
years = [str(year) for year in range(2017, 2022)]
coms = ["c72ef959b710612855fe4a03460d2bbd9d38fd7d",
        "94ef2027bed84f7335b5f3aa37f3fec99eb04089",
        "2c204246567cebcb5badf11550cc2778aa9b79a9",
        "5db135e3bd4e66e1c5e9301914fc7c33eed7f4a7",
        "6b538c395c9acaa3a472a634dad1ff651e3a375d"]

commits = dict()

for i in range(len(years)):
    commits[years[i]] = coms[i]
    
cmd_line = "cd ~/Documents/vamos2022/kops && git reset --hard "

### Search for keywords

In [4]:
def get_keywords():
    exp = "[N|n]ew.*,*Bool*"

    with open("./kops/pkg/featureflag/featureflag.go", 'r') as f:
        catch_feat = re.findall(exp, f.read())

    keywords = [feature.split('"')[1] for feature in catch_feat]

    # removing "Azure" because it affects more than 100 files, probably a bug 
    if "Azure" in keywords:
        keywords.remove("Azure")

    return [k.lower() for k in keywords]

get_keywords()

['cachenodeidentityinfo',
 'dnsprecreate',
 'enablelaunchtemplates',
 'enableexternalcloudcontroller',
 'enableexternaldns',
 'enableseparateconfigbase',
 'experimentalclusterdns',
 'googlecloudbucketacl',
 'keeplaunchconfigurations',
 'skipterraformformat',
 'specoverrideflag',
 'spotinst',
 'spotinstocean',
 'spotinsthybrid',
 'spotinstcontroller',
 'vfsvaultsupport',
 'vpcskipenablednssupport',
 'skipetcdversioncheck',
 'terraformjson',
 'legacyiam',
 'clusteraddons',
 'useserviceaccountiam',
 'publicjwks']

### Analyse the FT interferences

In [5]:
for year in years:
    
    print("\n------------\nSTART ANALYSING FOR YEAR", year,"\n------------\n")
    
    os.system(cmd_line+commits[year])
    keywords = get_keywords()
    
    print("\n--\nKeywords\n--\n")
    print(keywords)
    
    print("\n--\nFiles\n--\n")

    go_folders = [x[0] for x in os.walk("./kops/")]

    go_files = []
    for dir_name in go_folders:
        files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
        go_files.extend(files)

    go_file_interests = []

    for file in go_files:
        s = ""
        with open(file, "r") as f:
            s+=f.read().lower()+"\n"
        kw_file = [k for k in keywords if k in s]
        if len(kw_file) > 0 and "featureflag" in s:
            print(file, "contains :")
            for kw in kw_file:
                print("->",kw)
            go_file_interests.append(file)
            print("\n")


    print("\n--\nAnalyse FT\n--\n")

    for gfi in go_file_interests:

        s = ""
        with open(gfi, "r") as f:
            s+=f.read()+"\n"

        print("File :            ", gfi, "\n")

        source = bytes(s, "utf8")
        ast = parser.parse(source)

        tree = Tree()

        root_node = ast.root_node

        type_nodes = dict()

        def get_name(node):
            return source[node.start_byte:node.end_byte].decode('utf8').lower()

        def get_id(node):
            global type_nodes
            node_type = node.type
            if node_type not in type_nodes:
                type_nodes[node_type]=1
            else:
                type_nodes[node_type]+=1
            return node_type+str(type_nodes[node_type])

        def process(root_id, node):
            global tree
            node_id = get_id(node)
            node_name = get_name(node)
            for kw in keywords:
                if kw.lower() in node_name and node.type == 'if_statement':
                    #print("\n\n"+get_name(node)+"\n\n")
                    for c in node.children:
                        if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                            print("->",get_name(c))
            tree.create_node(node_name,
                             node_id,
                             parent = root_id)
            if len(node.children) != 0:
                for i in range(len(node.children)):
                    process(node_id, node.children[i])

        tree.create_node("root", "root")
        for i in range(len(root_node.children)):
            process("root", root_node.children[i])
        print("\n")

        #tree.show()


------------
START ANALYSING FOR YEAR 2017 
------------


--
Keywords
--

['dnsprecreate']

--
Files
--

./kops/upup/pkg/fi/cloudup/dns.go contains :
-> dnsprecreate


./kops/pkg/featureflag/featureflag.go contains :
-> dnsprecreate



--
Analyse FT
--

File :             ./kops/upup/pkg/fi/cloudup/dns.go 

-> !featureflag.dnsprecreate.enabled()


File :             ./kops/pkg/featureflag/featureflag.go 




------------
START ANALYSING FOR YEAR 2018 
------------


--
Keywords
--

['dnsprecreate', 'drainandvalidaterollingupdate', 'vpcskipenablednssupport', 'skipterraformformat', 'vspherecloudprovider', 'enableexternaldns', 'enableexternalcloudcontroller', 'enableseparateconfigbase', 'specoverrideflag']

--
Files
--

./kops/upup/pkg/fi/cloudup/dns.go contains :
-> dnsprecreate


./kops/upup/pkg/fi/cloudup/bootstrapchannelbuilder.go contains :
-> enableexternaldns
-> enableexternalcloudcontroller


./kops/upup/pkg/fi/cloudup/terraform/hcl_printer.go contains :
-> skipterraformformat



./kops/cmd/kops/rollingupdatecluster.go contains :
-> drainandvalidaterollingupdate


./kops/cmd/kops/create_cluster.go contains :
-> enableseparateconfigbase
-> specoverrideflag
-> spotinst
-> vspherecloudprovider



--
Analyse FT
--

File :             ./kops/upup/pkg/fi/cloudup/template_functions.go 

-> featureflag.enableexternalcloudcontroller.enabled()
-> featureflag.spotinst.enabled()
-> err == nil


File :             ./kops/upup/pkg/fi/cloudup/apply_cluster.go 

-> featureflag.spotinst.enabled()


File :             ./kops/upup/pkg/fi/cloudup/dns.go 

-> !featureflag.dnsprecreate.enabled()


File :             ./kops/upup/pkg/fi/cloudup/bootstrapchannelbuilder.go 

-> featureflag.enableexternaldns.enabled()
-> featureflag.spotinst.enabled()
-> featureflag.enableexternalcloudcontroller.enabled() && b.cluster.spec.externalcloudcontrollermanager != nil


File :             ./kops/upup/pkg/fi/cloudup/template_functions_test.go 



File :             ./kops/upup/pkg/fi/cloudup/terr



File :             ./kops/upup/pkg/fi/cloudup/dns.go 

-> !featureflag.dnsprecreate.enabled()


File :             ./kops/upup/pkg/fi/cloudup/new_cluster.go 

-> featureflag.spotinst.enabled()
-> opt.spotinstproduct != ""
-> opt.spotinstorientation != ""


File :             ./kops/upup/pkg/fi/cloudup/alitasks/launchconfiguration.go 

-> featureflag.keeplaunchconfigurations.enabled()


File :             ./kops/upup/pkg/fi/cloudup/terraform/target.go 

-> featureflag.terraformjson.enabled()


File :             ./kops/upup/pkg/fi/cloudup/bootstrapchannelbuilder/bootstrapchannelbuilder.go 

-> b.clusteraddons != nil
-> featureflag.publicjwks.enabled()
-> externaldns == nil || !externaldns.disable
-> b.useserviceaccountiam()
-> featureflag.enableexternaldns.enabled()
-> featureflag.spotinst.enabled() && featureflag.spotinstcontroller.enabled()
-> featureflag.spotinst.enabled() && featureflag.spotinstcontroller.enabled()


File :             ./kops/upup/pkg/fi/cloudup/awsup/aws_cloud.go